# Transaction and index

In [1]:
%reload_ext pycozo.ipyext_direct
%cozo_auth tutorial *******

It is true that stored relations and triples look like diametrically opposed concepts in Cozo. With triples you have to set up elaborate schema before use. With stored relations you just store your results, without even needing to pre-declare anything. Yet they are actually designed to work together. The intended way that they work together is through transactions.

We have already met transactions when we learned how to insert and mutate data in the triple store. But transactions can in fact contain many more things than `put` and `retract` requests. Let's first have the following schema in place:

In [2]:
:schema

:put ap {
    code: string unique
}

:put rt {
    src: ref,
    dst: ref,
    distance: float
}

,attr_id,op
0,10000030,assert
1,10000031,assert
2,10000032,assert
3,10000033,assert


Now let's try to add some data, but with a check:

In [3]:
:tx

{rt.src: {ap.code: 'A'}, rt.dst: {ap.code: 'B'}, rt.distance: -100}

:after {
    ?[dist] := [r rt.distance dist], dist < 0
    :assert none
}

eval::assert_none_failure

  × Triple store transaction failed as a post-condition failed
  ╰─▶ The query is asserted to return no result, but a tuple [-100.0] is found
   ╭─[6:1]
 6 │     ?[dist] := [r rt.distance dist], dist < 0
 7 │     :assert none
   ·     ────────────
 8 │ }
   ╰────


We have used the transaction directive `:after` signifying we want the query inside its braces to be checked _after_ the main transaction is done. In the `:after` block itself, we used the directive `:assert none` to indicate the above query should return no results at all. The error message also shows us the offending tuple.

We can check that in this case, no triple is actually inserted, since the transaction failed:

In [4]:
?[r, dist] := [r rt.distance dist]

,r,dist


Besides `:after`, there is also the `:before` directive, running before any triple transactions. In the block itself, the function `assert()` can also be very useful in the body of rules. See the manual for more information.

You may have any number of `:put`, `:retract`, `:before` or `:after`. They can be interleaved. When a transaction is run, all `:before` blocks are run in the order they are defined, then all `:put` and `:retract`, and finally all `:after` blocks are run.

The real power of the transaction is that you can put anything in your `:before` and `:after` blocks, including puts or retracts to stored relations!